# Differential Privacy for PRETSA

In [2]:
!git clone https://github.com/cmalvr/PRETSA

Cloning into 'PRETSA'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 195 (delta 42), reused 32 (delta 14), pack-reused 115 (from 1)
Receiving objects: 100% (195/195), 11.45 MiB | 16.24 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [5]:
!pip install -r /content/PRETSA/requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for anytree: filename=anytree-2.4.3-py2.py3-none-any.whl size=26707 sha256=ea97bdcfe1bef7270eac45fc07920ef46392d574b40f5486f94fa515213e5e52
  Stored in directory: /root/.cache/pip/wheels/f3/d3/8f/9eea88a7fe27ca18e7ebfc23e1012514c8b13a4552aa43be12
Successfully built anytree


In [14]:
!rm -r "/content/PRETSA/baselogs"

In [12]:
from pathlib import Path
import zipfile

# Define paths
zip_path = Path("/content/PRETSA/baselogs.zip")
extract_dir = Path("/content/PRETSA")

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Search for CSV files
csv_files = list(extract_dir.glob("*.csv"))

if csv_files:
    print("Found CSV files:", [f.name for f in csv_files])
else:
    print("No CSV files found.")

No CSV files found.
